# Format the CF data into the SWDE
website = domain

In [1]:
from markuplmft.examples.fine_tunning.run_swde import prepare_data

ModuleNotFoundError: No module named 'markuplmft.examples'

In [1]:
import pandas as pd
import pandavro as pdx
from ast import literal_eval
from pathlib import Path
import os   
import sys
import shutil

# Load

In [2]:
data_path = "../../../web-annotation-extractor/data/processed/train/dataset.avro"
df = pdx.read_avro(data_path)
df.annotations = df.annotations.apply(literal_eval)
len(df)

219173

In [3]:
df.head()

,url,domain,content_type,annotations,html,crawl_index,source,source_excel,source_batch
0,https://15gifts.com/get-in-touch/,15gifts.com,text/html,{'OFFICE_LOCATION': [{'text': 'Century House 1...,"<!doctype html>\n<html lang=""en-GB"" class=""no-...",s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-06-02 (BT-IT2)/Templates/15GIFTS ...,Batch - 2021-06-02 (BT-IT2)
1,https://www.3sisecurity.com/,3sisecurity.com,text/html,"{'INDUSTRY': [{'text': 'ATM', 'value': 'Bankin...","<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",s3://globality-raw/crawl-indexes/crawl-aug-31-...,cloudfactory,Batch - 2021-03-12 (S-B7)/Templates/3SI SECURI...,Batch - 2021-03-12 (S-B7)
2,https://www.3sisecurity.com/case-study-founder...,3sisecurity.com,text/html,{'CASE_STUDY': [{'text': 'the challenge atms t...,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",s3://globality-raw/crawl-indexes/crawl-aug-31-...,cloudfactory,Batch - 2021-05-19 (AI)/Templates/3sisecurity....,Batch - 2021-05-19 (AI)
3,https://www.3sisecurity.com/case-study-helping...,3sisecurity.com,text/html,{'CASE_STUDY': [{'text': 'the number of conven...,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",s3://globality-raw/crawl-indexes/crawl-aug-31-...,cloudfactory,Batch - 2021-05-19 (AI)/Templates/3sisecurity....,Batch - 2021-05-19 (AI)
4,https://www.3sisecurity.com/case-study-norther...,3sisecurity.com,text/html,{'CASE_STUDY': [{'text': 'there are unique hur...,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",s3://globality-raw/crawl-indexes/crawl-aug-31-...,cloudfactory,['Batch - 2021-03-12 (S-B7)/Templates/3SI SECU...,"['Batch - 2021-03-12 (S-B7)', 'Batch - 2021-05..."


# Remove hiphen from domains

In [4]:
df.domain = df.domain.apply(lambda x: x.replace('-', ''))

## Number of Domains

In [5]:
# df.iloc[0]['annotations']['PAST_CLIENT'].append({'text':'AIMORE', 'value':None})
df.domain.value_counts()

opentext.com             1462
progress.com             1197
blackbaud.com            1191
everbridge.com           1139
eng.it                   1139
                         ... 
depirrogarrone.com          1
mobileiron.com              1
entreehealth.com            1
optimizely.com              1
cormispartnership.com       1
Name: domain, Length: 493, dtype: int64

## Number of Pages

In [6]:
# Get pages with at least one annotation (just for simplification for now)
df = df[df['annotations'].apply(lambda x: len(x)>0)]
len(df)

11641

In [7]:
# df[df['annotations_len'] > 1]['domain'].value_counts().index.tolist()

In [8]:
# Assert that domain don't contain parenthesis
assert len(df[df['domain'].apply(lambda x: '(' in x or ')' in x)]) == 0
# Remove parenthesis from domain in case there is (assertion above fails)
# ... df = df[~df['domain'].apply(lambda x: '(' in x or ')' in x)]

In [9]:
annotation_tags = ["PAST_CLIENT", "OFFICE_LOCATION", "CASE_STUDY"][:1]
annotation_tags

['PAST_CLIENT']

In [10]:
# df['annotations_len'] = df['annotations'].apply(lambda x: len(x.get("PAST_CLIENT", '')))

# Remove annotations that cannot be found in the html

In [11]:
for tag in annotation_tags:
    print(f'- {tag}:')
    df[f'annotations-{tag}'] = df['annotations'].apply(lambda x: x.get(tag))        
    df[f'text-{tag}'] = df.dropna(subset=[f'annotations-{tag}'])[f'annotations-{tag}'].apply(lambda x: [y['text'] for y in x])    
    print(f"Before: {sum(df[f'text-{tag}'].dropna().apply(len))}")
    df[f'text-{tag}'] = df.dropna(subset=[f'text-{tag}']).apply(lambda row: ([annotation for annotation in row[f'text-{tag}'] if annotation in row['html']]), axis=1)
    print(f"After: {sum(df[f'text-{tag}'].dropna().apply(len))}")
    df = df[df[f'text-{tag}'].fillna('').apply(list).apply(len) > 0]

- PAST_CLIENT:
Before: 12660
After: 11058


In [12]:
df

,url,domain,content_type,annotations,html,crawl_index,source,source_excel,source_batch,annotations-PAST_CLIENT,text-PAST_CLIENT
4,https://www.3sisecurity.com/case-study-norther...,3sisecurity.com,text/html,{'CASE_STUDY': [{'text': 'there are unique hur...,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",s3://globality-raw/crawl-indexes/crawl-aug-31-...,cloudfactory,['Batch - 2021-03-12 (S-B7)/Templates/3SI SECU...,"['Batch - 2021-03-12 (S-B7)', 'Batch - 2021-05...","[{'text': 'Northern Tool & Equipment', 'value'...",[Northern Tool & Equipment]
8,https://www.3sisecurity.com/check-out-what-cum...,3sisecurity.com,text/html,{'PAST_CLIENT': [{'text': 'Cumberland Valley N...,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",s3://globality-raw/crawl-indexes/crawl-aug-31-...,cloudfactory,Batch - 2021-03-12 (S-B7)/Templates/3SI SECURI...,Batch - 2021-03-12 (S-B7),"[{'text': 'Cumberland Valley National Bank', '...",[Cumberland Valley National Bank]
10,https://www.3sisecurity.com/why-use-gps-becaus...,3sisecurity.com,text/html,{'CASE_STUDY': [{'text': 'organized retail cri...,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,['Batch - 2021-03-12 (S-B7)/Templates/3SI SECU...,"['Batch - 2021-03-12 (S-B7)', 'Batch - 2021-05...","[{'text': 'Sally’s Beauty', 'value': None}]",[Sally’s Beauty]
58,https://resources.aprimo.com/success-stories/-...,aprimo.com,text/html,"{'PAST_CLIENT': [{'text': 'Ubisoft', 'value': ...","<!DOCTYPE html>\n<html itemscope itemtype=""htt...",s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-03-12 (S-B8)/Templates/To be uplo...,Batch - 2021-03-12 (S-B8),"[{'text': 'Ubisoft', 'value': 'http://graph.gl...",[Ubisoft]
59,https://resources.aprimo.com/success-stories/a...,aprimo.com,text/html,"{'PAST_CLIENT': [{'text': 'AT&T', 'value': 'ht...","<!DOCTYPE html>\n<html itemscope itemtype=""htt...",s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-03-12 (S-B8)/Templates/To be uplo...,Batch - 2021-03-12 (S-B8),"[{'text': 'AT&T', 'value': 'http://graph.globa...",[AT&T]
...,...,...,...,...,...,...,...,...,...,...,...
11631,https://yesler.com/b2b-marketing-case-studies/...,yesler.com,text/html,{'CASE_STUDY': [{'text': 'when product marketi...,<!doctype html>\n<!--[if lt IE 7]> <html class...,s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-06-17 (MK1)/Templates/Yesler.xlsx,Batch - 2021-06-17 (MK1),"[{'text': 'Microsoft', 'value': 'http://graph....",[Microsoft]
11632,https://yesler.com/b2b-marketing-case-studies/...,yesler.com,text/html,{'CASE_STUDY': [{'text': 'hey did you know you...,<!doctype html>\n<!--[if lt IE 7]> <html class...,s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-06-17 (MK1)/Templates/Yesler.xlsx,Batch - 2021-06-17 (MK1),"[{'text': 'Zillow', 'value': 'http://graph.glo...",[Zillow]
11637,https://www.zerofox.com/resources/ac-milan-cas...,zerofox.com,text/html,"{'PAST_CLIENT': [{'text': 'AC Milan', 'value':...","<!doctype html>\n<html class=""no-js "" lang=""en...",s3://globality-raw/crawl-indexes/full-upload-j...,cloudfactory,Batch - 2021-03-12 (S-B3)/Templates/ZEROFOX IN...,Batch - 2021-03-12 (S-B3),"[{'text': 'AC Milan', 'value': None}]",[AC Milan]
11638,https://www.zerofox.com/resources/civil-aviati...,zerofox.com,text/html,{'PAST_CLIENT': [{'text': 'The Civil Aviation ...,"<!doctype html>\n<html class=""no-js "" lang=""en...",s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-03-12 (S-B3)/Templates/ZEROFOX IN...,Batch - 2021-03-12 (S-B3),"[{'text': 'The Civil Aviation Authority', 'val...",[The Civil Aviation Authority]


# Format and Save data

In [13]:
raw_data_folder = Path.cwd().parents[2] / 'swde/my_CF_sourceCode'

if os.path.exists(raw_data_folder):
    print(f'Are you sure you want to remove this folder? (y/n) \n{raw_data_folder}')
    answer = input()    
    if answer == 'y':
        try:
            shutil.rmtree(raw_data_folder)
            print(f"REMOVED: {raw_data_folder}")
        except OSError as e:
            print ("Error: %s - %s." % (e.filename, e.strerror))
else:
    print(f"File '{groundtruth_data_path}' not found in the directory")
    
domains = list(df.domain.value_counts().index)

groundtruth_data_path = raw_data_folder / 'groundtruth' / 'WAE'   
groundtruth_data_path.mkdir(parents=True, exist_ok=True)


# domains  = [x for x in domains if x in ['cosmostreet.com',
#  'bt.com',
#  'sidebench.com',
#  'ccl.org',
#  'ipc.com',
#  'interiorarchitects.com',
#  'commodorebuilders.com',
#  'hardingconsultants.co.nz',
#  'solarwinds.com',
#  'monotype.com',
#  'digitalaccessibilitycentre.org',
#  'humankind.nz',
#  'phonecoach.com',
#  'elearningbrothers.com']]

for e, domain in enumerate(domains):        
    df_domain = df[df.domain == domain]
    
    print(f"{e:>3}: {len(df_domain):>5} page(s) - {domain:>25}")
    
    domain_annotations = {}    
    
    page_count = 0
    domain_len = len(df_domain)
    
    for enum, df_page in df_domain.iterrows():
        # Save html
        html = df_page['html']
        raw_data_path = raw_data_folder / 'WAE' / f"WAE-{domain}({domain_len})"
        raw_data_path.mkdir(parents=True, exist_ok=True)
        raw_data_path = (raw_data_path / str(page_count).zfill(4)).with_suffix('.htm')
        print(raw_data_path)
        
        Html_file = open(raw_data_path, "w")
        Html_file.write(html)
        Html_file.close()
        
        page_count += 1
                
        # Get groundtruth for page for each tag
        for tag in annotation_tags:            
            domain_annotations[tag] = domain_annotations.get(tag, [])
            if not df_page.isnull()[f'text-{tag}']:
                annotations = df_page[f'text-{tag}']                
                # Remove image links from text annotation
                annotate = [annotation.strip() if (annotation and 'http' not in annotation.strip()) else '' for annotation in annotations]
            else:
                annotate = []
            # print(f'annotate: \n{annotate} - {len(annotate)}')            
            domain_annotations[tag].append(annotate)            
        print()
        # if raw_data_path.name == '0042.htm':
        #     break
        
    # Save groundtruth    
    for tag, page_annotations in domain_annotations.items():
        groundtruth_data_tag_path = groundtruth_data_path / f"WAE-{domain}-{tag}.txt"
        print(groundtruth_data_tag_path)

        page_annotations_df = pd.DataFrame(page_annotations)
        
        # Count number of annotations
        page_annotations_df['number of values'] = page_annotations_df.T.count()        
        
        # Invert columns order 
        cols = page_annotations_df.columns.tolist()
        page_annotations_df = page_annotations_df[cols[::-1]] 
        
        # Get page index
        page_annotations_df.reset_index(inplace=True)
        page_annotations_df['index'] = page_annotations_df['index'].apply(lambda x: str(x).zfill(4))
        
        # Add one extra row on the top
        page_annotations_df.loc[-1] = page_annotations_df.count()  # adding a row
        page_annotations_df.index = page_annotations_df.index + 1  # shifting index
        page_annotations_df = page_annotations_df.sort_index()
        
        page_annotations_df.to_csv(groundtruth_data_tag_path, sep="\t", index=False)

Are you sure you want to remove this folder? (y/n) 
/data/GIT/swde/my_CF_sourceCode


 y


REMOVED: /data/GIT/swde/my_CF_sourceCode
  0:   208 page(s) -                  hitt.com
/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0000.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0001.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0002.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0003.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0004.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0005.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0006.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0007.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0008.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0009.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0010.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0011.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0012.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(208)/0013.htm

/data/GIT/swde/m

In [19]:

print(p)
raw_data_folder == p

/data/GIT/swde/my_CF_sourceCode


True

---